### AES-128

明文：16字节，密钥16字节，轮数10，AES的运算是定义在$GF(2^8)$上的，其中乘法和加法操作如下：

* 加法：按位异或
* 乘法：num乘以00000010左移一位，若num最高位为1，则移位的结果再与00011011异或；更高次可重复使用上述过程得到结果。

**加密流程**

第0轮：明文与第0轮密钥做加密操作（轮密钥加）

第1-9轮：明文与第1-9轮密钥做加密操作（字节代替、行移位、列混淆、轮密钥加）

第10轮：明文与第10轮密钥做加密操作（字节代替、行移位、列混淆）

**密钥的扩展**

密钥被扩展为11个16字节的轮密钥，各用于每一轮的加密

**明文与密钥的表示**

16个字节按列排列为4 * 4的字节矩阵

**各个加密操作**

* 字节代替变换——
  * S盒变换：对状态中的每个字节，高四位作为行，低四位作为列，从S盒中去对应字节作为输出
  * 逆S盒变换类似，代替表不同
  * S盒的构造

* 行移位变换——
  * 正向行移位：状态第一、二、三、四行分别循环左移0、1、2、3个字节
  * 逆向则相反，即循环右移

* 列混淆变换——
  * 正向列混淆：对每列独立的进行操作，如图，其中乘法与加法都是定义在GF(2^8)上的。

  <img src="https://s1.ax1x.com/2018/11/15/ivImyF.md.png" width=450>

  * 逆向列混淆类似，矩阵如下：

  <img src="https://s1.ax1x.com/2018/11/15/ivI3Jx.md.png" width=450>

* 轮密钥加变换——

  * 正向/逆向：128位的状态按位与128位的轮密钥异或

**密钥扩展算法**

* 用输入密钥（16个字节，4个字）直接作为扩展密钥数组的前4个字
* 对之后的每个字$w[i]$，依赖于$w[i-1]$和$w[i-4]$
  - 若 $i \ mod \ 4 \ne 0$ ， 则$w[i] = w[i-1] ⊕ w[i-4]$
  - 否则，$w[i] = G(w[i-1])⊕w[i-4] $
  其中，G的操作如下：
  - 字循环：把四个字节循环左移一个字节
  - 字代替：使用S盒代替4个字节
  - 字代替的结果与轮常量$RC[j]$异或得到输出，其中轮常量是一个字，右边三个字节为0。$RC[j]$每轮都不同，$RC[j] = 2 * RC[j-1],RC[1]=1$


In [1]:
import numpy

##### $GF(2^8)$下字节的加法与乘法

In [14]:
def GF_sum(num1, num2):
    return num1 ^ num2

def GF_mul(num1, num2):
    res, tmp = 0, num1
    i = 1
    while i <= 128:
        res = res ^ (0 if (num2 & i) == 0 else tmp)
        i *= 2
        if tmp & 0b10000000: tmp = (tmp * 2) % 256 ^ 0b00011011
        else: tmp = (tmp * 2) % 256
    return res

In [15]:
GF_mul(0b01010111, 0b10000011)

0b1010111
0b10101110
0b1000111
0b10001110
0b111
0b1110
0b11100
0b111000


193